# Analyse the Outlets in the Dataset by Mankoff (2020)

This notebook loads the outlets from the dataset by Mankoff (2020), selects the ones near the 79° North Glacier (79NG), and shows them on a map of Northeast Greenland.
The map also shows the bathymetry and topography of the area, obtained from the RTopo-2.0.4 dataset.

To use this notebook:
1. Download the [dataset by Mankoff (2020)](https://doi.org/10.22008/FK2/XKQVL7) into a folder `data/Mankoff_2020_v5` (the large NetCDF files are not needed here and can be left out).
2. Download the file `RTopo-2.0.4_30sec_Greenland.nc` from the [dataset by Schaffer et al. (2019)](https://doi.org/10.1594/PANGAEA.905295) into a folder `data`.
3. Download the [GitHub repository GEUS-Glaciology-and-Climate/freshwater](https://github.com/GEUS-Glaciology-and-Climate/freshwater) into a folder `freshwater`.

References:
* Mankoff, Kenneth et al. (2020): _Greenland liquid water discharge from 1958 through 2019,_ Earth System Science Data, https://doi.org/10.5194/essd-12-2811-2020 (paper)
* Mankoff, Kenneth (2020): _Streams, Outlets, Basins, and Discharge [k=1.0],_ GEUS Dataverse, V5 (release 2023), https://doi.org/10.22008/FK2/XKQVL7 (dataset)
* Mankoff, Kenneth (6 October 2023): _Freshwater,_ GitHub, https://github.com/GEUS-Glaciology-and-Climate/freshwater (repository)
* Schaffer, Janin et al. (2019): _An update to Greenland and Antarctic ice sheet topography, cavity geometry, and global bathymetry (RTopo-2.0.4),_ PANGAEA, https://doi.org/10.1594/PANGAEA.905295 (dataset)

Notebook by Markus Reinert (IOW, 2023–2024, https://orcid.org/0000-0002-3761-8029).

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cmocean
from pyproj import Transformer
from shapely.geometry.multipolygon import MultiPolygon

from freshwater.discharge import discharge as Discharge

## Set up the coordinate transformation

In [ ]:
transform_xy_to_lonlat = Transformer.from_crs("EPSG:3413", "EPSG:4326", always_xy=True).transform
transform_lonlat_to_xy = Transformer.from_crs("EPSG:4326", "EPSG:3413", always_xy=True).transform

## Load the bathymetry data

In [ ]:
rtopo = xr.open_dataset("data/RTopo-2.0.4_30sec_Greenland.nc")
rtopo = rtopo.set_index(londim="lon", latdim="lat")
rtopo = rtopo.rename({"latdim": "lat", "londim": "lon"})
rtopo = rtopo.sel(lon=slice(-29, -18), lat=slice(78.6, 80.3))

# Compute and add Cartesian coordinates
X, Y = transform_lonlat_to_xy(*xr.broadcast(rtopo.lon, rtopo.lat))
rtopo.coords["x"] = (["lon", "lat"], X, {"long_name": "Easting", "units": "m"})
rtopo.coords["y"] = (["lon", "lat"], Y, {"long_name": "Northing", "units": "m"})

rtopo

## Load the outlet data

Whether we include upstream ice outlets or not, loading the outlet data takes essentially the same time.
Therefore, we always load upstream outlets, but we can decide later to neglect them by using `outlets[~outlets.upstream]`.

In [ ]:
# Choose between 79NG (True) or Zachariae Isstrom (False)
SELECT_79NG = True
if SELECT_79NG:
    position = "-22,79.4"
else:
    position = "-22,79.0"
outlets = Discharge("data/Mankoff_2020_v5/freshwater/", position, upstream=True, quiet=False).outlets()
print(f"Loaded {len(outlets)} outlets with {len(set(outlets.id))} unique IDs.")
outlets

## Do basic checks and selections

### Verify the values of `upstream` and `domain`

In [ ]:
assert len(outlets[~outlets.upstream]) == 2, "there are not exactly 2 selected (non-upstream) outlets"
assert all(outlets[~outlets.upstream].domain == ["land", "ice"]), "the 2 selected (non-upstream) outlets are not land and ice"
assert all(outlets[outlets.upstream].domain == "ice"), "not all upstream outlets are ice"

### Extract the selected ice and land outlets

This works because of the checks above.

In [ ]:
outlet_ice = outlets[(outlets.domain == "ice") & ~outlets.upstream].iloc[0]
outlet_land = outlets[outlets.domain == "land"].iloc[0]

### Check the coast coordinates

In [ ]:
# Check that coast coordinates of selected ice outlet are equal to coordinates of land outlet
assert outlet_ice.coast_x == outlet_land.x and outlet_ice.coast_lon == outlet_land.lon
assert outlet_ice.coast_y == outlet_land.y and outlet_ice.coast_lat == outlet_land.lat

# Check that the land outlet has no coast coordinates
assert np.isnan(outlet_land.coast_lon) and np.isnan(outlet_land.coast_lat)

### Test the coordinate transformation

In [ ]:
lon, lat = transform_xy_to_lonlat(outlets.x, outlets.y)
assert np.allclose(lon, outlets.lon) and np.allclose(lat, outlets.lat)
x, y = transform_lonlat_to_xy(outlets.lon, outlets.lat)
assert np.allclose(x, outlets.x) and np.allclose(y, outlets.y)

## Show outlets and bathymetry

In [ ]:
# Choose between Cartesian coordinates (x,y: True) and ellipsoidal coordinates (lat,lon: False)
XY = True

coords = {"x": "x", "y": "y"} if XY else {"x": "lon", "y": "lat"}
convert = lambda x, y: (x, y) if XY else transform_xy_to_lonlat(x, y)

def plot_outlet(outlet, kwargs={}):
    l, = plt.plot(outlet[coords["x"]], outlet[coords["y"]], **kwargs)
    kwargs = {"color": l.get_color(), "ls": l.get_linestyle(), "lw": l.get_linewidth()}
    if isinstance(outlet.basin, MultiPolygon):
        for geom in outlet.basin.geoms:
            plt.plot(*convert(*geom.exterior.xy), **kwargs)
    else:
        plt.plot(*convert(*outlet.basin.exterior.xy), **kwargs)

plt.figure(dpi=300)

# Show the selected land and ice outlets
plot_outlet(outlet_land, {"color": "k", "marker": "s", "label": "land outlet and basin boundary"})
plot_outlet(outlet_ice, {"color": "r", "marker": "o", "label": "ice outlet and basin boundary", "ls": "--"})

# Show the upstream outlets
n_upstream = 0
for i, outlet in outlets[outlets.upstream].iterrows():
    if outlet.id == outlet_ice.id:
        print(f"Skipping outlet {i} with same ID as the selected ice outlet.")
        continue
    plot_outlet(outlet, {"marker": ".", "lw": 0.5})
    n_upstream += 1
plt.plot([], [], ".-", lw=0.5, color="gray", label=f"{n_upstream} upstream ice outlets")
plt.legend(loc="upper left")

if XY:
    plt.gca().set_aspect("equal")
    plt.xlim(0.32e6, 0.55e6)
    plt.ylim(-1.12e6, -0.95e6)
rtopo.bedrock_topography.plot(**coords, cmap=cmocean.cm.topo, alpha=1/2, cbar_kwargs={"label": "Bathymetry from RTopo [m]"})
plt.title(f"Outlets and Basins around {'79NG' if SELECT_79NG else 'ZI'}")

filename = f"figures/outlets_{'79NG' if SELECT_79NG else 'ZI'}{'_xy' if XY else ''}.png"
print("Saving as", filename)
plt.savefig(filename)